<a href="https://colab.research.google.com/github/srijit43/NLP-theory-and-code/blob/main/05_10TF_IDF_vs_Count_Vect_GradBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building Machine Learning Classifiers: Evaluate Gradient Boosting with GridSearchCV

**Grid-search:** Exhaustively search all parameter combinations in a given grid to determine the best model.

**Cross-validation:** Divide a dataset into k subsets and repeat the holdout method k times where a different subset is used as the holdout set in each iteration.

### Read in text

In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string
nltk.download('stopwords')

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

# TF-IDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_tfidf_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)

# CountVectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['body_text'])
X_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count.toarray())], axis=1)

X_count_feat.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,49,4.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,62,3.2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,28,7.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,135,4.4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


###Adjustment

In [2]:
X_tfidf_feat = X_tfidf_feat.rename(str,axis='columns')
X_count_feat = X_count_feat.rename(str,axis='columns')

### Exploring parameter settings using GridSearchCV

In [3]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

In [4]:
gb = GradientBoostingClassifier()
param = {
    'n_estimators': [100, 150],
    'max_depth': [7, 11, 15],
    'learning_rate': [0.1]
}

clf = GridSearchCV(gb, param, cv=5, n_jobs=-1)
cv_fit = clf.fit(X_tfidf_feat, data['label'])
pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
5,412.242332,57.536589,0.226898,0.036610,0.1,15,150,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.964991,0.975763,0.971249,0.967655,0.968553,0.969642,0.003656,1
3,338.544765,1.057375,0.272551,0.068152,0.1,11,150,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.964991,0.976661,0.971249,0.968553,0.964960,0.969283,0.004380,2
0,149.684952,0.472211,0.229406,0.007929,0.1,7,100,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.964991,0.977558,0.972147,0.965858,0.965858,0.969283,0.004870,3
1,225.288865,0.448600,0.246622,0.029936,0.1,7,150,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.967684,0.977558,0.969452,0.964960,0.965858,0.969102,0.004502,4
4,280.153295,2.891118,0.263187,0.039181,0.1,15,100,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.964991,0.975763,0.968553,0.967655,0.966757,0.968744,0.003702,5


In [ ]:
gb = GradientBoostingClassifier()
param = {
    'n_estimators': [50, 100, 150],
    'max_depth': [7, 11, 15],
    'learning_rate': [0.1]
}

clf = GridSearchCV(gb, param, cv=5, n_jobs=-1)
cv_fit = clf.fit(X_count_feat, data['label'])
pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_learning_rate,param_max_depth,param_n_estimators,params,rank_test_score,split0_test_score,...,split2_test_score,split2_train_score,split3_test_score,split3_train_score,split4_test_score,split4_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
5,341.866984,0.335839,0.969463,1.000000,0.1,11,150,"{'n_estimators': 150, 'learning_rate': 0.1, 'm...",1,0.965022,...,0.970350,1.0,0.963163,1.0,0.970350,1.000000,3.450213,0.048011,0.005320,0.00000
7,335.012637,0.264707,0.969283,1.000000,0.1,15,100,"{'n_estimators': 100, 'learning_rate': 0.1, 'm...",2,0.965022,...,0.967655,1.0,0.964960,1.0,0.972147,1.000000,7.205083,0.027251,0.004513,0.00000
2,213.266939,0.279388,0.968385,0.999955,0.1,7,150,"{'n_estimators': 150, 'learning_rate': 0.1, 'm...",3,0.965919,...,0.968553,1.0,0.960467,1.0,0.966757,0.999775,2.651567,0.022217,0.006508,0.00009
8,356.912155,0.183561,0.968385,1.000000,0.1,15,150,"{'n_estimators': 150, 'learning_rate': 0.1, 'm...",3,0.962332,...,0.967655,1.0,0.963163,1.0,0.971249,1.000000,45.305546,0.031481,0.005594,0.00000
4,241.101819,0.263556,0.968205,1.000000,0.1,11,100,"{'n_estimators': 100, 'learning_rate': 0.1, 'm...",5,0.963229,...,0.970350,1.0,0.961366,1.0,0.968553,1.000000,3.736889,0.026132,0.005716,0.00000
